In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [55]:
# load training and test data
train = pd.read_csv('data/train_processed_new.csv')
test = pd.read_csv('data/test_processed_new.csv')

In [56]:
# preprocess skills feature

# determine unique skills
skills = set()
for m in train.skills:
    skills.update(g for g in m.split('|'))
skills = sorted(skills)

#make a column for each skill
for skill in skills:
    train[skill] = [int(skill in s.split('|')) for s in train.skills]

In [57]:
# do same calculation for test set as well
for skill in skills:
    test[skill] = [int(skill in s.split('|')) for s in test.skills]

In [58]:
# determine problem tags
problem_tags = set()
for tag1, tag2, tag3, tag4, tag5 in zip(train.tag1, train.tag2, train.tag3, train.tag4, train.tag5):
    problem_tags.update([tag1, tag2, tag3, tag4, tag5])
problem_tags = sorted(problem_tags)

In [59]:
problem_tags[0] = 'Not Specified'

In [60]:
# make a column for each problem type
for problem_tag in problem_tags:
    train[problem_tag] = [int(problem_tag in tags) for tags in train[['tag1', 'tag2', 'tag3', 'tag4', 'tag5']].values]

In [10]:
for problem_tag in problem_tags:
    test[problem_tag] = [int(problem_tag in tags) for tags in test[['tag1', 'tag2', 'tag3', 'tag4', 'tag5']].values]

In [61]:
# balanced dataset
%run scripts/helper.py
train_sample = balanced_sample(train)

In [62]:
# response variable ( sample )
y_sample = train_sample.solved_status

In [63]:
# response variable
y = train.solved_status

## Modelling

In [74]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [77]:
model1 = build_extreme_gradient_boosting(train, test)
model2 = build_logistic_regression_model(train, test)
model3 = build_random_forest_classifier(train, test)
# model = build_logistic_regression_model(train, test)
# model = build_knn_classifier(train, test)
# model = build_extra_trees_classifier(train, test)

In [78]:
scores = eval_models([model1, model2, model3], train, y)

accuracy score: 0.877000
accuracy score: 0.860700
accuracy score: 0.872017
combined score: 0.875983
accuracy score: 0.877200
accuracy score: 0.862500
accuracy score: 0.872850
combined score: 0.876000
accuracy score: 0.877433
accuracy score: 0.862783
accuracy score: 0.873200
combined score: 0.876650
accuracy score: 0.384350
accuracy score: 0.860200
accuracy score: 0.871483
combined score: 0.867467
accuracy score: 0.877950
accuracy score: 0.861467
accuracy score: 0.872783
combined score: 0.876450



In [79]:
print 'Mean: %f and Standard Deviation: %f' %(np.mean(scores), np.std(scores))

Mean: 0.874510 and Standard Deviation: 0.003531


In [34]:
predsTrain, predsTest, y_train, y_test = analyze_output(train)


Accuracy on the training set 0.834863 
Accuracy on the test set 0.819851 


In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, predsTest)

In [ ]:
misclassified = train.ix[(y_test != predsTest).index.values]
misclassified['preds'] = predsTest

In [ ]:
misclassified_class_0 = misclassified[misclassified.solved_status == 0]

In [ ]:
misclassified_class_0[['accuracy', 'solved_status', 'preds']]

In [ ]:
sns.pointplot(x="solved_status", y="accuracy", data=misclassified);

In [71]:
# train model
import time

start_time = time.time()

# model1.fit(train, y)
# model2.fit(train, y)
# model3.fit(train, y)
model.fit(train, y)

elapsed_time = time.time() - start_time

print 'It took %f seconds to train the model ' %(elapsed_time)

It took 290.984000 seconds to train the model 


In [72]:
# predictions1  = model1.predict(test)
# predictions2 = model2.predict(test)
# predictions3 = model3.predict(test)

# ensemble_preds = np.array(majority_voting([predictions1, predictions2, predictions3]))
predictions = model.predict(test)

In [73]:
# submissions
submission_df = pd.DataFrame({'Id': test.id.values, 'solved_status': predictions.astype(int)})
submission_df.to_csv('submissions/thirty_eighth_xgb_num_attempts_at_problem.csv', index=False)